In [1]:
import os
import tensorflow.compat.v1 as tf
from tensorflow.compat.v1 import keras
from tensorflow.compat.v1.keras import layers,datasets,utils,models
from tensorflow.compat.v1.keras.preprocessing.image import ImageDataGenerator
from tensorflow.compat.v1.keras.models import Sequential, Model
from tensorflow.compat.v1.keras.layers import Dense, Dropout, Activation, Flatten,Input
from tensorflow.compat.v1.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalMaxPooling2D
import numpy as np
from tensorflow.compat.v1.keras.utils import to_categorical
from tensorflow.compat.v1.keras.regularizers import l2

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.keras.backend.set_session(tf.Session(config=config))

In [4]:
global_parameters = {'batch_size': 32,
                    'num_classes': 10,
                    'epochs': 30,
                     'leraning_rate': 0.0001,
                    'data_augmentation': True,
                     'user_dropout': True,
                     'dropout_rate': 0.1,
                     'user_l2': False,
                    'l2_lambda': 0.0001}

In [4]:
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.0
x_test /= 255.0
y_train = to_categorical(y_train, global_parameters['num_classes'])
y_test = to_categorical(y_test, global_parameters['num_classes'])

In [6]:
def conv_model():
    regularizer = None
    if global_parameters['user_l2']:
        regularizer = l2(global_parameters['l2_lambda'])
    inp = Input((32, 32, 3))
    x = Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizer)(inp)
    x = Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizer)(x)
    x = MaxPooling2D()(x)
    
    x = Conv2D(128, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizer)(x)
    x = Conv2D(128, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizer)(x)
    x = MaxPooling2D()(x)
    
    x = Conv2D(256, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizer)(x)
    x = Conv2D(256, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizer)(x)
    x = MaxPooling2D()(x)
    
    x = Flatten()(x)
    if global_parameters['user_dropout']:
        x = Dropout(0.10)(x)
    out = Dense(10, activation='softmax')(x)
    return Model(inputs=inp, outputs=out)

In [7]:
model = conv_model()
# opt = keras.optimizers.Adam(lr=global_parameters['leraning_rate'])
opt = keras.optimizers.SGD(lr=global_parameters['leraning_rate'])
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 128)         0     

In [9]:
model.load_weights('./model/impovementVGG16_weigths.h5')

In [10]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='.\\Graph3', histogram_freq=0, write_graph=True, write_images=True)

In [11]:
if not global_parameters['data_augmentation']:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=global_parameters['batch_size'],
              epochs=global_parameters['epochs'],
              validation_data=(x_test, y_test),
              shuffle=True, callbacks=[tbCallBack])
else:
    print('Using real-time data augmentation.')
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    
    datagen.fit(x_train)
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=global_parameters['batch_size']),
                        steps_per_epoch=x_train.shape[0] // global_parameters['batch_size'],
                        epochs=global_parameters['epochs'],
                        validation_data=(x_test, y_test), callbacks=[tbCallBack])

Using real-time data augmentation.
Epoch 1/30
1562/1562 [==============================] - 60s 38ms/step - loss: 0.0783 - accuracy: 0.9726 - val_loss: 0.6155 - val_accuracy: 0.8697
Epoch 2/30
1562/1562 [==============================] - 59s 38ms/step - loss: 0.0768 - accuracy: 0.9731 - val_loss: 0.6043 - val_accuracy: 0.8714
Epoch 3/30
1562/1562 [==============================] - 65s 41ms/step - loss: 0.0800 - accuracy: 0.9715 - val_loss: 0.6067 - val_accuracy: 0.8709
Epoch 4/30
1562/1562 [==============================] - 64s 41ms/step - loss: 0.0807 - accuracy: 0.9721 - val_loss: 0.6044 - val_accuracy: 0.8709
Epoch 5/30
1562/1562 [==============================] - 62s 40ms/step - loss: 0.0776 - accuracy: 0.9723 - val_loss: 0.6036 - val_accuracy: 0.8714
Epoch 6/30
1562/1562 [==============================] - 64s 41ms/step - loss: 0.0781 - accuracy: 0.9723 - val_loss: 0.6007 - val_accuracy: 0.8711
Epoch 7/30
1562/1562 [==============================] - 63s 40ms/step - loss: 0.0775 - ac

In [12]:
model.save_weights('./model/impovementVGG16_weigths.h5')